In [ ]:
import pandas as pd
import json
import openai
with open('character_label.json') as f:
    data_list = []
    data = json.load(f)
    for i in range(50) :
        token = data[i]['tokens']
        label = data[i]['ner_tags']
        data_list.append((token, label))
data_list

[(['오',
   '프',
   '커',
   '피',
   '에',
   '서',
   '아',
   '메',
   '리',
   '카',
   '노',
   '핫',
   '으',
   '로',
   '2',
   '잔',
   '주',
   '문',
   '해',
   '주',
   '세',
   '요'],
  [1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0]),
 (['롤',
   '키',
   '에',
   '서',
   '치',
   '즈',
   '롤',
   '케',
   '이',
   '크',
   '한',
   '조',
   '각',
   '이',
   '랑',
   '아',
   '메',
   '리',
   '카',
   '노',
   '아',
   '이',
   '스',
   '로',
   '하',
   '나',
   '씩',
   '부',
   '탁',
   '드',
   '립',
   '니',
   '다'],
  [1,
   2,
   0,
   0,
   3,
   4,
   4,
   4,
   4,
   4,
   5,
   6,
   6,
   0,
   0,
   3,
   4,
   4,
   4,
   4,
   0,
   0,
   0,
   0,
   5,
   6,
   6,
   0,
   0,
   0,
   0,
   0,
   0]),
 (['리',
   '스',
   '아',
   '라',
   '비',
   '카',
   '에',
   '서',
   '라',
   '떼',
   '에',
   '디',
   '카',
   '페',
   '인',
   '옵',
   '션',
   '으',
   '로',
   '두',
   '잔',
   ',',
   '라',
   '지',
   '사',
   '이',
   '즈',
   '로',
   '부',
   '탁',
   '드',
   '릴',
   '게',
   '요'],
  [1,
   2,
  

In [ ]:
def generate_gpt_answer(setence, ner_tags):
    # 프롬프트 작성
    prompt = (
        f"""
        이제부터 너는 음식을 주문 문장에 NER 태깅을 하는 프로젝트의 담당자가 되어서 주어진 예시와 같은 라벨링을 수행하는 역할을 할거야.

        태깅 규칙은 BIO 태깅을 참고해서 아래와 같이 정했어.
        B-store는 매장명의 시작 글자, I-store는 매장명의 중간 글자, B-menu는 음식명의 시작 글자, I- menu는 음식명의 중간 글자, 
        B-count는 주문 수량의 시작 글자 , I-count는 주문 수량의 중간 글자야. 그 외에 상관 없는 글자들은 0으로 설정했어

        숫자로 나타내면 아래와 같아.
        그 외: 0 , B-store : 1 , I- store : 2 , B-menu : 3 , I- menu : 4 , 수량: B-count : 5 , I-count : 6

        태깅은 아래를 참고해서 실시하면 돼. 아래 파일의 각 원소는 주문 문장과 각 글자에 맞는 라벨로 이루어져 있어.
        
        "tokens" : ["유", "가", "네", "한", "우", "곰", "탕", "에", "서", "한", "우", "곰", "탕", "두", "그", "릇", ",", "밥", "두", "공", "기", "추", "가", "요", "."],
        "ner_tags" : [1, 2, 2, 2, 2, 2, 2, 0, 0, 3, 4, 4, 4, 5, 6, 6, 0, 3, 5, 6, 6, 0, 0, 0, 0]

        "tokens" : ["순", "대", "국", "밥", "하", "나", "랑", "순", "대", "추", "가", ",", "본", "래", "순", "대", "에", "서", "주", "문", "할", "게", "요", "."],
        "ner_tags" : [3, 4, 4, 4, 5, 6, 0, 3, 4, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]

        "tokens" : ["김", "치", "백", "서", "에", "서", "김", "치", "찌", "개", "두", "그", "릇", "이", "랑", "밥", "두", "공", "기", "부", "탁", "해", "요", "."],
        "ner_tags" : [1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 5, 6, 6, 0, 0, 3, 5, 6, 6, 0, 0, 0, 0, 0]

        "tokens" : ["김", "형", "제", "고", "기", "의", "철", "학", "에", "서", "삼", "겹", "살", "4", "0", "0", "그", "람", ",", "된", "장", "찌", "개", "하", "나", "추", "가", "할", "게", "요", "."],
        "ner_tags" : [1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 4, 4, 5, 6, 6, 6, 6, 0, 3, 4, 4, 4, 5, 6, 0, 0, 0, 0, 0, 0]

        "tokens" : ["샤", "브", "마", "니", "아", "에", "서", "샤", "브", "샤", "브", "2", "인", "분", "이", "랑", "공", "깃", "밥", "세", "개", "주", "세", "요", "."],
        "ner_tags" : [1, 2, 2, 2, 2, 0, 0, 3, 4, 4, 4, 5, 6, 6, 0, 0, 3, 4, 4, 5, 6, 0, 0, 0, 0]

        "tokens" : ["김", "치", "찜", "하", "나", "랑", "공", "깃", "밥", "세", "개", ",", "부", "뚜", "막", "김", "치", "찜", "에", "서", "주", "문", "할", "게", "요", "."],
        "ner_tags" : [3, 4, 4, 5, 6, 0, 3, 4, 4, 5, 6, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]

        "tokens" : ["내", "가", "닭", "도", "리", "탕", "이", "다", "에", "서", "닭", "도", "리", "탕", "1", "인", "분", "과", "밥", "두", "개", "추", "가", "해", "주", "세", "요", "."],
        "ner_tags" : [1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 4, 4, 4, 5, 6, 6, 0, 3, 5, 6, 0, 0, 0, 0, 0, 0, 0]

        "tokens" : ["기", "대", "만", "족", "에", "서", "불", "고", "기", "덮", "밥", "하", "나", "랑", "치", "킨", "마", "요", "덮", "밥", "하", "나", "부", "탁", "드", "립", "니", "다", "."],
        "ner_tags" : [1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 5, 6, 0, 3, 4, 4, 4, 4, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0]

        "tokens" : ["칼", "국", "수", "두", "그", "릇", ",", "고", "기", "추", "가", "로", "현", "풍", "닭", "칼", "국", "수", "에", "서", "주", "문", "할", "게", "요", "."],
        "ner_tags" : [3, 4, 4, 5, 6, 6, 0, 3, 4, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]

        "tokens" : ["해", "장", "국", "두", "그", "릇", "이", "랑", "만", "두", ",", "방", "일", "해", "장", "국", "에", "서", "주", "문", "할", "게", "요", "."],
        "ner_tags" : [3, 4, 4, 5, 6, 6, 0, 0, 3, 4, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]

        네가 NER 라벨링을 수행할 문장은 아래와 같아.
        {sentence}
        {setence}에 라벨을 붙일거야 너는 이 {ner_tags} 결과를 바꿔주는것을 수행하면 돼
        태깅 규칙에 잘 맞춰서 한글자씩 숫자로 변환한 BIO태깅 부탁해
        다른 설명은 다 제외하고 그냥 주어진 문장과 라벨링 결과만 반환 해줘.
        ~~~와 같습니다. 이런 설명은 다 없이
        ["스","타","벅","스","에","서","아","메","리","카","노", "다", "섯", "잔", "주", "문", "해", "주", "세", "요"]
        [1,2,2,2,0,0,3,4,4,4,4,5,6,6,0,0,0,0,0,0]
        이런식으로 리스트만 반환해주면 돼 친절할 필요 없어 그냥 답만줘.
        주문 문장 리스트의 원소 개수랑 라벨링 문장 리스트 원소 개수는 같아야 한다는것 잊으면 안돼. 
        니가 결과 값으로 반환하는 태깅 리스트 원소 개수는 입력으로 들어오는 리스트인 {ner_tags}의 원소 개수랑 똑같아야 한다는걸 잊지마. 개수가 안맞는 경우가 많더라
        매장 이름이 앞에 오는 경우랑 뒤에 오는 경우 두가지로 나뉘어서 데이터가 있으니까 이 부분도 신경써서 라벨링해줘.
        값을 
        ```json
        ``` 이걸로 묶으면 안돼 문장리스트와 답변리스트만 보여줘
        """
    )

    # API 호출
    response = openai.chat.completions.create(
        model = "gpt-4o-mini", # 모델 선택
        messages = [{"role" : "system", "content" : "당신은 NER태깅 프로젝트 담당자입니다."}, # 시스템에게 역할 부여
                    {"role" : "user", "content" : prompt}], # 사용자는 prompt 객체를 시스템에게 전달
        temperature = 0, # 창의성 정도
    )
    return response

with open('original_label.json') as f2:
    data2 = json.load(f2)
    for i in range(len(data2)) :
        sentence = data2[i]['tokens']
        ner_tags = data2[i]['ner_tags']

        response = generate_gpt_answer(sentence, ner_tags)
        result_text = response.choices[0].message.content

        with open('gpt_labeling_4.txt', mode='a') as f:
            f.write(result_text + '\n')